In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint

from passlib.hash import bcrypt
from neo4j import GraphDatabase

from datetime import date, datetime
from collections import defaultdict

In [3]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "rachsam99"), encrypted = False)
db_session = driver.session()

In [4]:
client = MongoClient()
db = client.package_adbms
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'package_adbms')


In [5]:
class User:
    def __init__(self, userid):
        self.userid = userid
    
    def find(self):
        
        query = "MATCH(u : User {id : $userid}) RETURN (u)"
        if db_session.run(query, userid = self.userid).single() != None:
            query = "MATCH(u : User {id : $userid}) RETURN (u.password)"
            return(db_session.run(query, userid = self.userid).single().value())
        else:
            return(None)
        
    def verify_password(self, password):
        user_found = self.find()
        if user_found:
            return(bcrypt.verify(password, user_found))
        else:
            return False
    
    def set_password(self, username, password):
        user_found = self.find()
        if user_found:
            return False
        else:
            password = bcrypt.encrypt(password)
            query = "CREATE (u: User {id: $userid, name: $username, password : $password}) "
            db_session.run(query, userid = self.userid, username = username, password = password)
            return True

In [6]:
class Host:
    def __init__(self, hostid):
        self.hostid = hostid
    
    def find(self):
        query = "MATCH(h : Hosts {host_id : $hostid}) RETURN (h)"
        if db_session.run(query, hostid = self.hostid).single() != None:
            query = "MATCH(h : Hosts {host_id : $hostid}) RETURN (h.password)"
            return(db_session.run(query, hostid = self.hostid).single().value())
        else:
            return(None)
        
    def verify_password(self, password):
        host_found = self.find()
        if host_found:
            return(bcrypt.verify(password, host_found))
        else:
            return False
    
    def set_password(self, hostname, password):
        host_found = self.find()
        if host_found:
            return False
        else:
            password = bcrypt.encrypt(password)
            query = "CREATE (h: Hosts {host_id: $hostid, host_name: $hostname, password : $password}) "
            db_session.run(query, hostid = self.hostid, hostname = hostname, password = password)
            return True
        
    def add_details(self, hostname, image_url, about, identity_verified, location):
        host = {
            "host_id"             : self.hostid,
            "host_name"           : hostname,
            "host_image"          : image_url,
            "about"               : about,
        
            "listings_count"      : int(0),
        
            "response_time"       : "NA",
            "acceptance_rate"     : "NA",
            "response_rate"       : "NA",
        
            "host_since"          : datetime.strptime(str(datetime.now()).split(" ")[0],"%Y-%m-%d"),
            "is_superhost"        : "f",
            "host_location"       : location,
        
            "identity_verified"   : identity_verified     
        }
        db.hosts.insert_one(host)
    
    def display_hostprofile(self):
        host_information = db.hosts.find_one({"host_id" : self.hostid})
        
        listings = dict()
        
        query = "MATCH (h: Hosts {host_id : $hostid}) -[:HOSTS]-> (l: Listing) RETURN l.listing_id, l.listing_name"
        for record in db_session.run(query, hostid = self.hostid):
            details_l = db.listings.find_one({"listing_id" : record["l.listing_id"]})
            del details_l['_id']
            
            listings[record['l.listing_name']] = details_l
            
        return(host_information, listings)
    
    def add_listings_details(self, listing_details):
        db.listings.insert_one(listing_details)
        #Adding Listing Node
        query = """CREATE (l: Listing {host_id: $host_id, listing_id : $listing_id
                                    , listing_name : $listing_name, neighbourhood : $neighbourhood})"""
        
        db_session.run(query, host_id = listing_details["host_id"], listing_id = listing_details["listing_id"], listing_name = listing_details["listing_name"], neighbourhood = listing_details["neighbourhood"])
        
        #Creating Relationship between listing and address
        query = """
            MATCH (l: Listing), (a: Address {neighbourhood : $neighbourhood})
            WHERE l.listing_id = $listing_id, l.neighbourhood = $neighbourhood, a.neighbourhood = $neighbourhood
            CREATE (l) -[:LOCATED]-> (a)
        """
        db_session.run(query, listing_id = listing_details["listing_id"], neighbourhood = listing_details["neighbourhood"])

In [7]:
class Address:
    def __init__(self, neighbourhood):
        self.neighbourhood = neighbourhood
    
    def check_location(self):
        query = "MATCH(a : Address {neighbourhood : $neighbourhood}) RETURN (a)"
        if db_session.run(query, neighbourhood = self.neighbourhood).single() != None:
            return(True)
        return(None)
    
    def get_reviews(self, l_id):
        details = []
        query_review = "MATCH (r: Review) -[:REVIEWED]-> (l: Listing {listing_id : $l_id}) RETURN r.reviewer_id,  r.comments, r.date"
        for record_r in db_session.run(query_review, l_id = l_id):
            details_r = {'reviewer_id' : record_r['r.reviewer_id'], 'review' : record_r['r.comments'], 'date' : str(record_r['r.date'])}
            details.append(details_r)
        return(details)
    
    def get_listings(self, requirements):
        
        query = {"neighbourhood" : self.neighbourhood}
        start_date = requirements["start_date"]
        end_date = requirements["end_date"]
        
        #For Dates
        if(start_date and end_date):
            d0 = datetime.strptime(start_date, "%Y-%m-%d")
            d1 = datetime.strptime(end_date, "%Y-%m-%d")
            if(start_date < end_date):
                delta = d1 - d0
                delta = int(str(delta).split(" ")[0])
                
                if(delta <= 30):
                    query["availability_30"] = {"$gte" : delta}
                elif(delta >= 31 and delta <=60):
                    query["availability_60"] = {"$gte" : delta}
                elif(delta >=61 and delta <= 90):
                    query["availability_90"] = {"$gte" : delta}
                elif(delta >= 91 and delta <= 365):
                    query["availability_365"] = {"$gte" : delta}
                    
                
        elif(start_date):
            query["availability_30"] = {"$gte" : 1}
            
        elif(end_date):
            d0 = datetime.strptime(str(datetime.now()).split(" ")[0],"%Y-%m-%d")
            d1 = datetime.strptime(end_date, "%Y-%m-%d")
            if(start_date < end_date):
                delta = d1 - d0
                delta = int(str(delta).split(" ")[0])
                if(delta <= 30):
                    query["availability_30"] = {"$gte" : delta}
                elif(delta >= 31 and delta <=60):
                    query["availability_60"] = {"$gte" : delta}
                elif(delta >=61 and delta <= 90):
                    query["availability_90"] = {"$gte" : delta}
                elif(delta >= 91 and delta <= 365):
                    query["availability_365"] = {"$gte" : delta}
                    
        #For Guests
        if(requirements["guests"]):
            guests = int(requirements["guests"])
            query["no_of_accommodates"] = {"$gte" : guests}
            
        listings = dict()
        reviews = dict()
            
        for record in db.listings.find(query):
            details_l = db.listings.find_one({"listing_id" : record["listing_id"]})
            del details_l['_id']
            listings[record['listing_name']] = details_l
            reviews[record['listing_id']] = self.get_reviews(record['listing_id'])
                    
        return(listings, reviews)          
       
    def available_filters(self, filters):
        filter_dict = {}
        filter_dict["neighbourhood"] = self.neighbourhood
        
        if(filters[0] != "Cancellation Policy"):
                filter_dict["cancellation_policy"] = filters[0]
        if(filters[1] != "Property Type"):
                filter_dict["property_type"] = filters[1]
        if(filters[2] != "Instant Bookable"):
            if(filters[2] == "Yes"):
                filter_dict["instant_bookable"] = 't'
            elif(filters[2] == "No"):
                filter_dict["instant_bookable"] = 'f'
        if(filters[3] != "Room Type"):
                filter_dict["room_type"] = filters[3]
        if(filters[4] != "Bed Type"):
                filter_dict["bed_type"] = filters[4]
        if(filters[5] != "Price"):
                price = filters[5]
                start_price = float(price.split("-")[0][1:])
                end_price = float(price.split("-")[1][1:])
                filter_dict["price"] = {"$gte" : start_price, "$lte" : end_price}
                
        start_date = filters[6]
        end_date = filters[7]
        
        #For Dates
        if(start_date and end_date):
            d0 = datetime.strptime(start_date, "%Y-%m-%d")
            d1 = datetime.strptime(end_date, "%Y-%m-%d")
            if(start_date < end_date):
                delta = d1 - d0
                delta = int(str(delta).split(" ")[0])
                
                if(delta <= 30):
                    filter_dict["availability_30"] = {"$gte" : delta}
                elif(delta >= 31 and delta <=60):
                    filter_dict["availability_60"] = {"$gte" : delta}
                elif(delta >=61 and delta <= 90):
                    filter_dict["availability_90"] = {"$gte" : delta}
                elif(delta >= 91 and delta <= 365):
                    filter_dict["availability_365"] = {"$gte" : delta}
                    
                
        elif(start_date):
            filter_dict["availability_30"] = {"$gte" : 1}
            
        elif(end_date):
            d0 = datetime.strptime(str(datetime.now()).split(" ")[0],"%Y-%m-%d")
            d1 = datetime.strptime(end_date, "%Y-%m-%d")
            if(start_date < end_date):
                delta = d1 - d0
                delta = int(str(delta).split(" ")[0])
                if(delta <= 30):
                    filter_dict["availability_30"] = {"$gte" : delta}
                elif(delta >= 31 and delta <=60):
                    filter_dict["availability_60"] = {"$gte" : delta}
                elif(delta >=61 and delta <= 90):
                    filter_dict["availability_90"] = {"$gte" : delta}
                elif(delta >= 91 and delta <= 365):
                    filter_dict["availability_365"] = {"$gte" : delta}
                    
        #For Guests
        if(filters[8]):
            guests = int(filters[8])
            filter_dict["no_of_accommodates"] = {"$gte" : guests}
                
        return(filter_dict)
    
    def apply_filters(self, filter_dict):
        listings = dict()
        reviews = dict()
        
        for record in db.listings.find(filter_dict):
            del record["_id"]
            listings[record['listing_name']] = record
            reviews[record['listing_id']] = self.get_reviews(record['listing_id'])
            
        return(listings, reviews)

In [8]:
class Reviews:
    def __init__(self, listing_id, userid, comment):
        self.listing_id = listing_id
        self.userid = userid
        self.comment = comment
        
    def create_review(self):
        today = date.today()
        date_today = today.strftime("%d/%m/%Y")
        
        #Creating a Review Node
        query1 = "CREATE (r: Review {date : $date, listing_id : $listing_id, comments : $comment, reviewer_id : $reviewer_id})"
        db_session.run(query1, date = date_today, listing_id = self.listing_id, comment = self.comment, reviewer_id = self.userid)
        
        #Relationship betweem user and review node
        query2 = "MATCH (u: User), (r: Review) WHERE u.id = $userid AND r.reviewer_id = $userid AND r.listing_id = $listing_id CREATE (u) -[:WROTE]-> (r) RETURN u, r"
        db_session.run(query2, userid = self.userid, listing_id = self.listing_id)
        
        #Relationship between review and listing_node
        query3 = "MATCH (l: Listing), (r: Review) WHERE l.listing_id = $listing_id AND r.reviewer_id = $userid AND r.listing_id = $listing_id CREATE (r) -[:REVIEWED]-> (l)"
        db_session.run(query3, userid = self.userid, listing_id = self.listing_id)
        
    def delete_review(self):
        query1 = "MATCH (r:Review { reviewer_id : $reviewer_id, listing_id : $listing_id, comments : $comment}) DETACH DELETE r"
        db_session.run(query1, reviewer_id = self.userid, listing_id = self.listing_id, comment = self.comment)

In [9]:
class Recommendations:
    def __init__(self, userid):
        self.userid = userid
        
    def check_if_newuser(self):
        query = "MATCH p=(r: Review {reviewer_id : $userid})-[:REVIEWED]->() RETURN p LIMIT 1"
        if db_session.run(query, userid = self.userid).single() != None:
            return("No")
        return("Yes")
    
    def random_recommendations(self):
        listings = dict()
        reviews = dict()
        
        query = """
            MATCH (l: Listing) RETURN l.listing_id, l.listing_name LIMIT 7
        """
        for record in db_session.run(query):
            details_l = db.listings.find_one({"listing_id" : record["l.listing_id"]})
            del details_l['_id']
            
            listings[record['l.listing_name']] = details_l
            
            details = []
            query_review = "MATCH (r: Review) -[:REVIEWED]-> (l: Listing {listing_id : $l_id}) RETURN r.reviewer_id, r.comments, r.date"
            for record_r in db_session.run(query_review, l_id = record['l.listing_id']):
                details_r = {'reviewer_id' : record_r['r.reviewer_id'], 'review' : record_r['r.comments'], 'date' : str(record_r['r.date'])}
                details.append(details_r)
                    
            reviews[record['l.listing_id']] = details
            
        return(listings, reviews)
            
    def collaborative_filtering(self):
        listings = dict()
        reviews = dict()
        
        query = """MATCH (u: User {id : $userid}) -[:WROTE]-> (r: Review) -[:REVIEWED]-> (l: Listing)
        MATCH (l) <-[:REVIEWED]- (:Review) <-[:WROTE]- (other: User) -[:WROTE]-> (:Review) -[:REVIEWED]-> (res: Listing)
        RETURN res.listing_id, res.listing_name, count(*) AS SCORE ORDER BY SCORE DESC LIMIT 7"""
        
        if db_session.run(query, userid = self.userid).single() != None:
            for record in db_session.run(query, userid = self.userid):
                details_l = db.listings.find_one({"listing_id" : record["res.listing_id"]})
                del details_l['_id']
                listings[record['res.listing_name']] = details_l
            
                details = []
                query_review = "MATCH (r: Review) -[:REVIEWED]-> (l: Listing {listing_id : $l_id}) RETURN r.reviewer_id, r.comments, r.date"
                for record_r in db_session.run(query_review, l_id = record['res.listing_id']):
                    details_r = {'reviewer_id' : record_r['r.reviewer_id'], 'review' : record_r['r.comments'], 'date' : str(record_r['r.date'])}
                    details.append(details_r)
                    
                reviews[record['res.listing_id']] = details
        else:
            listings, reviews = self.contentbased_filtering()
            
        return(listings, reviews)
    
    def contentbased_filtering(self):
        listings = dict()
        reviews = dict()
        
        query = """ MATCH  (u: User {id: $userid}) -[:WROTE]-> (r: Review)-[:REVIEWED]->(l: Listing) -[:LOCATED]-> (a: Address)
                    WITH u, l, COLLECT(DISTINCT a.neighbourhood) AS neighborhoods 
                    MATCH (res)-[:LOCATED]->(a: Address)
                    WITH res, a, neighborhoods WHERE a.neighbourhood IN neighborhoods
                    RETURN res.listing_id, res.listing_name LIMIT 7
                """
        
        for record in db_session.run(query, userid = self.userid):
            details_l = db.listings.find_one({"listing_id" : record["res.listing_id"]})
            del details_l['_id']
            listings[record['res.listing_name']] = details_l
            
            details = []
            query_review = "MATCH (r: Review) -[:REVIEWED]-> (l: Listing {listing_id : $l_id}) RETURN r.reviewer_id, r.comments, r.date"
            for record_r in db_session.run(query_review, l_id = record['res.listing_id']):
                details_r = {'reviewer_id' : record_r['r.reviewer_id'],  'review' : record_r['r.comments'], 'date' : str(record_r['r.date'])}
                details.append(details_r)
                    
            reviews[record['res.listing_id']] = details
        return(listings, reviews)